In [1]:
import fiftyone as fo
from fiftyone import ViewField as F
import glob
from datetime import date

### Load dataset

In [2]:
# Load dataset
ds = fo.load_dataset('D2S')
ds

Name:           D2S
Media type:     image
Num samples:    24098
Persistent:     True
Info:           {}
Tags:           ['aug', 'byte', 'test', 'train', 'val', 'val_clutter', 'val_wo_occ', 'water']
Sample fields:
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    prediction:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

### Configure PATHs

In [3]:
# Edit PATHs
ANNOTATION_PATH = r'G:\My Drive\Food_Recognition\Datasets\D2S\annotations\D2S_training.json'
IMAGES_PATH = r'G:\My Drive\Food_Recognition\Datasets\D2S\Image_Data\images' 
OUTPUT_IMAGES_DIR = r'G:\My Drive\Food_Recognition\Datasets\YOLO_dataset'
# Check PATHs
ANNOTATION_PATH, IMAGES_PATH, OUTPUT_IMAGES_DIR

('G:\\My Drive\\Food_Recognition\\Datasets\\D2S\\annotations\\D2S_training.json',
 'G:\\My Drive\\Food_Recognition\\Datasets\\D2S\\Image_Data\\images',
 'G:\\My Drive\\Food_Recognition\\Datasets\\YOLO_dataset')

### Load and check JSON

In [4]:
# Load JSON
import json
with open(ANNOTATION_PATH) as json_file: 
    samples_anns = json.load(json_file)

In [5]:
# Samples amount
len(samples_anns['images'])

4379

In [6]:
# Check structure
samples_anns.keys()

dict_keys(['images', 'info', 'licenses', 'annotations', 'categories'])

In [7]:
# Check image's properties
samples_anns['images'][0].keys()

dict_keys(['file_name', 'height', 'id', 'license', 'width'])

In [8]:
# Check annotation's properties
samples_anns['annotations'][0].keys()

dict_keys(['segmentation', 'area', 'iscrowd', 'image_id', 'bbox', 'category_id', 'id'])

In [9]:
# Check category's properties
samples_anns['categories'][0].keys()

dict_keys(['supercategory', 'id', 'name'])

In [10]:
type(samples_anns['categories'][0]['name'])

str

### Upload train images to FiftyOne

In [11]:
#TAG = f'train_{date.today()}'
TAG = 'train'

In [13]:
def find_label_by_id(id):
    for cat in samples_anns['categories']:
        if cat['id'] == id:
            return cat['name']
    return None

In [14]:
for im in samples_anns['images']:
    filepath = IMAGES_PATH + '\\' + im['file_name']
    sample = fo.Sample(filepath=filepath)
    detections = []
    for ann in samples_anns['annotations']:
        if ann['image_id'] == im['id']:
            label = find_label_by_id(ann['category_id'])
            bounding_box = [float(cor) for cor in ann['bbox']]
            bounding_box[0], bounding_box[2] = bounding_box[0] / im['width'], bounding_box[2] / im['width']
            bounding_box[1], bounding_box[3] = bounding_box[1] /  im['height'], bounding_box[3] / im['height']
            print(bounding_box)
            detections.append(fo.Detection(label=label, bounding_box=bounding_box))
    sample['ground_truth'] = fo.Detections(detections=detections)
    sample['tags'] = [TAG, 'byte']
    ds.add_sample(sample)

[0.4203125, 0.03819444444444445, 0.23020833333333332, 0.2625]
[0.2640625, 0.06180555555555556, 0.23489583333333333, 0.27569444444444446]
[0.159375, 0.17847222222222223, 0.23229166666666667, 0.32222222222222224]
[0.175, 0.4076388888888889, 0.159375, 0.27152777777777776]
[0.20260416666666667, 0.5590277777777778, 0.23489583333333333, 0.3277777777777778]
[0.34270833333333334, 0.6784722222222223, 0.22708333333333333, 0.26319444444444445]
[0.49114583333333334, 0.6416666666666667, 0.23020833333333332, 0.27569444444444446]
[0.5953125, 0.48333333333333334, 0.23125, 0.32083333333333336]
[0.6541666666666667, 0.30833333333333335, 0.15729166666666666, 0.2722222222222222]
[0.5515625, 0.09861111111111111, 0.2359375, 0.3229166666666667]
[0.4203125, 0.03819444444444445, 0.23020833333333332, 0.2625]
[0.2640625, 0.06180555555555556, 0.23489583333333333, 0.27569444444444446]
[0.159375, 0.17847222222222223, 0.23229166666666667, 0.32222222222222224]
[0.175, 0.4076388888888889, 0.159375, 0.27152777777777776]

[0.6, 0.28958333333333336, 0.17864583333333334, 0.40625]
[0.49947916666666664, 0.12291666666666666, 0.3, 0.43819444444444444]
[0.3921875, 0.09027777777777778, 0.3255208333333333, 0.3104166666666667]
[0.26875, 0.08541666666666667, 0.3333333333333333, 0.3263888888888889]
[0.190625, 0.11944444444444445, 0.2947916666666667, 0.4409722222222222]
[0.2109375, 0.28541666666666665, 0.1765625, 0.4076388888888889]
[0.19010416666666666, 0.42291666666666666, 0.30052083333333335, 0.43819444444444444]
[0.2713541666666667, 0.5819444444444445, 0.32447916666666665, 0.3055555555555556]
[0.3875, 0.5715277777777777, 0.3296875, 0.3194444444444444]
[0.5026041666666666, 0.425, 0.2953125, 0.4305555555555556]
[0.6, 0.28958333333333336, 0.17864583333333334, 0.40625]
[0.49947916666666664, 0.12291666666666666, 0.3, 0.43819444444444444]
[0.3921875, 0.09027777777777778, 0.3255208333333333, 0.3104166666666667]
[0.26875, 0.08541666666666667, 0.3333333333333333, 0.3263888888888889]
[0.190625, 0.11944444444444445, 0.2947

[0.1765625, 0.10694444444444444, 0.346875, 0.48541666666666666]
[0.22291666666666668, 0.23958333333333334, 0.16979166666666667, 0.49166666666666664]
[0.196875, 0.36875, 0.30520833333333336, 0.5111111111111111]
[0.24739583333333334, 0.5409722222222222, 0.3796875, 0.3576388888888889]
[0.34739583333333335, 0.56875, 0.38645833333333335, 0.28958333333333336]
[0.4661458333333333, 0.3923611111111111, 0.34375, 0.47638888888888886]
[0.5953125, 0.25, 0.17135416666666667, 0.49166666666666664]
[0.5817708333333333, 0.08263888888888889, 0.26614583333333336, 0.2534722222222222]
[0.1515625, 0.2875, 0.3494791666666667, 0.2763888888888889]
[0.45989583333333334, 0.5861111111111111, 0.3776041666666667, 0.27847222222222223]
[0.4427083333333333, 0.0, 0.1765625, 0.26944444444444443]
[0.15520833333333334, 0.47152777777777777, 0.35104166666666664, 0.21597222222222223]
[0.5770833333333333, 0.30486111111111114, 0.25104166666666666, 0.49027777777777776]
[0.28020833333333334, 0.0, 0.14375, 0.30694444444444446]
[0.

[0.31145833333333334, 0.5652777777777778, 0.3, 0.38125]
[0.11458333333333333, 0.4638888888888889, 0.20677083333333332, 0.32222222222222224]
[0.3125, 0.11944444444444445, 0.2765625, 0.30416666666666664]
[0.4505208333333333, 0.5340277777777778, 0.26510416666666664, 0.3888888888888889]
[0.19583333333333333, 0.6138888888888889, 0.2859375, 0.3861111111111111]
[0.20625, 0.15069444444444444, 0.3125, 0.40069444444444446]
[0.5520833333333334, 0.42986111111111114, 0.2421875, 0.36527777777777776]
[0.384375, 0.7173611111111111, 0.2625, 0.2826388888888889]
[0.21041666666666667, 0.30694444444444446, 0.22916666666666666, 0.3472222222222222]
[0.5401041666666667, 0.2520833333333333, 0.3078125, 0.3972222222222222]
[0.53125, 0.6194444444444445, 0.28229166666666666, 0.3416666666666667]
[0.21614583333333334, 0.4076388888888889, 0.27447916666666666, 0.40902777777777777]
[0.5109375, 0.16111111111111112, 0.25364583333333335, 0.28055555555555556]
[0.63125, 0.3923611111111111, 0.28541666666666665, 0.38333333333

[0.2921875, 0.08194444444444444, 0.24270833333333333, 0.2916666666666667]
[0.234375, 0.6451388888888889, 0.22447916666666667, 0.32222222222222224]
[0.6385416666666667, 0.42291666666666666, 0.1921875, 0.18333333333333332]
[0.20104166666666667, 0.175, 0.2234375, 0.3236111111111111]
[0.40520833333333334, 0.7395833333333334, 0.2265625, 0.23333333333333334]
[0.5911458333333334, 0.18263888888888888, 0.21614583333333334, 0.25833333333333336]
[0.196875, 0.3638888888888889, 0.17447916666666666, 0.29375]
[0.553125, 0.6375, 0.23802083333333332, 0.2701388888888889]
[0.5067708333333333, 0.059722222222222225, 0.1578125, 0.2659722222222222]
[0.20833333333333334, 0.5145833333333333, 0.24166666666666667, 0.325]
[0.6479166666666667, 0.4326388888888889, 0.221875, 0.30694444444444446]
[0.3411458333333333, 0.04097222222222222, 0.17239583333333333, 0.28194444444444444]
[0.32916666666666666, 0.6465277777777778, 0.22291666666666668, 0.2423611111111111]
[0.6671875, 0.2111111111111111, 0.159375, 0.2840277777777

...

[0.24947916666666667, 0.2534722222222222, 0.2671875, 0.4305555555555556]
[0.25625, 0.3548611111111111, 0.29583333333333334, 0.4201388888888889]
[0.2921875, 0.45555555555555555, 0.3151041666666667, 0.3701388888888889]
[0.3484375, 0.45694444444444443, 0.31666666666666665, 0.3423611111111111]
[0.4359375, 0.3854166666666667, 0.2947916666666667, 0.40347222222222223]
[0.47239583333333335, 0.3055555555555556, 0.2671875, 0.4270833333333333]
[0.4354166666666667, 0.21319444444444444, 0.2953125, 0.41875]
[0.3807291666666667, 0.16597222222222222, 0.3151041666666667, 0.3680555555555556]
[0.3234375, 0.18194444444444444, 0.31875, 0.3548611111111111]
[0.259375, 0.1875, 0.29322916666666665, 0.4263888888888889]
[0.24947916666666667, 0.2534722222222222, 0.2671875, 0.4305555555555556]
[0.25625, 0.3548611111111111, 0.29583333333333334, 0.4201388888888889]
[0.2921875, 0.45555555555555555, 0.3151041666666667, 0.3701388888888889]
[0.3484375, 0.45694444444444443, 0.31666666666666665, 0.3423611111111111]
[0.435

[0.5317708333333333, 0.5701388888888889, 0.08385416666666666, 0.2638888888888889]
[0.29270833333333335, 0.38333333333333336, 0.171875, 0.2791666666666667]
[0.5151041666666667, 0.325, 0.20833333333333334, 0.17777777777777778]
[0.571875, 0.4652777777777778, 0.15572916666666667, 0.26944444444444443]
[0.3802083333333333, 0.4673611111111111, 0.075, 0.2777777777777778]
[0.453125, 0.25069444444444444, 0.215625, 0.16111111111111112]
[0.5671875, 0.3645833333333333, 0.20416666666666666, 0.18194444444444444]
[0.39114583333333336, 0.5097222222222222, 0.17864583333333334, 0.2701388888888889]
[0.3880208333333333, 0.17569444444444443, 0.17916666666666667, 0.26458333333333334]
[0.5177083333333333, 0.24930555555555556, 0.2078125, 0.15]
[0.4432291666666667, 0.5451388888888888, 0.215625, 0.15763888888888888]
[0.3625, 0.21944444444444444, 0.0765625, 0.27152777777777776]
[0.434375, 0.12777777777777777, 0.18020833333333333, 0.2569444444444444]
[0.4942708333333333, 0.46458333333333335, 0.21197916666666666, 0

[0.30572916666666666, 0.5354166666666667, 0.19166666666666668, 0.2569444444444444]
[0.19739583333333333, 0.275, 0.20572916666666666, 0.15486111111111112]
[0.5755208333333334, 0.3472222222222222, 0.16770833333333332, 0.2722222222222222]
[0.6078125, 0.41458333333333336, 0.16666666666666666, 0.25763888888888886]
[0.42552083333333335, 0.04097222222222222, 0.15885416666666666, 0.26875]
[0.5010416666666667, 0.04097222222222222, 0.18020833333333333, 0.2701388888888889]
[0.5296875, 0.33055555555555555, 0.17239583333333333, 0.23680555555555555]
[0.353125, 0.3034722222222222, 0.10729166666666666, 0.2826388888888889]
[0.5328125, 0.23055555555555557, 0.08020833333333334, 0.25555555555555554]
[0.30520833333333336, 0.4027777777777778, 0.190625, 0.23819444444444443]
[0.4296875, 0.19305555555555556, 0.1453125, 0.25555555555555554]
[0.33229166666666665, 0.5361111111111111, 0.2078125, 0.10416666666666667]
[0.3307291666666667, 0.25833333333333336, 0.19166666666666668, 0.1763888888888889]
[0.3838541666666

[0.5953125, 0.45694444444444443, 0.11979166666666667, 0.15833333333333333]
[0.37916666666666665, 0.28541666666666665, 0.12395833333333334, 0.16458333333333333]
[0.4083333333333333, 0.7048611111111112, 0.12864583333333332, 0.17430555555555555]
[0.584375, 0.32569444444444445, 0.1203125, 0.15833333333333333]
[0.33645833333333336, 0.35, 0.1234375, 0.17222222222222222]
[0.5411458333333333, 0.6729166666666667, 0.13697916666666668, 0.16319444444444445]
[0.515625, 0.2222222222222222, 0.12291666666666666, 0.1625]
[0.33177083333333335, 0.43819444444444444, 0.12083333333333333, 0.17152777777777778]
[0.6348958333333333, 0.53125, 0.13385416666666666, 0.1625]
[0.41510416666666666, 0.19375, 0.12083333333333333, 0.16180555555555556]
[0.3609375, 0.5145833333333333, 0.125, 0.1625]
[0.6557291666666667, 0.3388888888888889, 0.12552083333333333, 0.17569444444444443]
[0.3260416666666667, 0.24930555555555556, 0.11822916666666666, 0.1597222222222222]
[0.42135416666666664, 0.5506944444444445, 0.1265625, 0.15972

[0.303125, 0.16458333333333333, 0.17239583333333333, 0.24513888888888888]
[0.6151041666666667, 0.5340277777777778, 0.15416666666666667, 0.15763888888888888]
[0.2203125, 0.29583333333333334, 0.19583333333333333, 0.22013888888888888]
[0.6291666666666667, 0.37569444444444444, 0.15885416666666666, 0.13125]
[0.21302083333333333, 0.4618055555555556, 0.20104166666666667, 0.22152777777777777]
[0.5729166666666666, 0.2013888888888889, 0.13333333333333333, 0.1701388888888889]
[0.28541666666666665, 0.5631944444444444, 0.18020833333333333, 0.2638888888888889]
[0.4708333333333333, 0.11180555555555556, 0.096875, 0.19652777777777777]
[0.40572916666666664, 0.6055555555555555, 0.1625, 0.2951388888888889]
[0.3333333333333333, 0.15694444444444444, 0.11770833333333333, 0.18472222222222223]
[0.5145833333333333, 0.5729166666666666, 0.17239583333333333, 0.28125]
[0.221875, 0.3159722222222222, 0.15, 0.1326388888888889]
[0.5703125, 0.47847222222222224, 0.2, 0.2361111111111111]
[0.20625, 0.47638888888888886, 0.1

### Check your train images

In [20]:
train = ds.match_tag(TAG)
train

Dataset:        D2S
Media type:     image
Num samples:    4379
Tags:           ['byte', 'water']
Sample fields:
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    prediction:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
Pipeline stages:
    1. MatchTag(tag='water')

### Save dataset

In [18]:
ds.persistent = True
ds.save()

### Extract categories

In [14]:
CATEGORIES = ['orange_single']

OUTPUT_TRAIN_DIR = r'G:\My Drive\Food_Recognition\Datasets & Images\D2S\Image_Data\train'
OUTPUT_VAL_DIR = r'G:\My Drive\Food_Recognition\Datasets & Images\D2S\Image_Data\val'

In [15]:
filtered = ds.filter_labels(
    "ground_truth",
    F("label").is_in(CATEGORIES),
    only_matches=True,
)
filtered

Dataset:        D2S
Media type:     image
Num samples:    1447
Tags:           ['aug', 'byte', 'test', 'train', 'val', 'val_clutter', 'water']
Sample fields:
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    prediction:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
Pipeline stages:
    1. FilterLabels(field='ground_truth', filter={'$in': ['$$this.label', [...]]}, only_matches=True)

### Convert categorial images to TFRecords

In [ ]:
EXPORT_DIR = r'G:\My Drive\Food_Recognition\Datasets\export_dir'
DATASET_TYPE = fo.types.TFObjectDetectionDataset
filtered.export(export_dir=EXPORT_DIR, dataset_type=DATASET_TYPE)

### Convert categorial images to JSON

In [100]:
EXPORT_DIR = r'G:\My Drive\Food_Recognition\Datasets\export_dir'
DATASET_TYPE = fo.types.COCODetectionDataset
filtered.export(export_dir=EXPORT_DIR, dataset_type=DATASET_TYPE)

Directory 'G:\My Drive\Food_Recognition\Datasets & Images\export_dir' already exists; export will be merged with existing files
 100% |███████| 119/119 [29.9s elapsed, 0s remaining, 2.7 samples/s]       


### Convert categorial images to YOLO

In [26]:
EXPORT_DIR = r'G:\My Drive\Food_Recognition\Datasets\export_dir'
DATASET_TYPE = fo.types.YOLODataset
filtered.export(export_dir=EXPORT_DIR, dataset_type=DATASET_TYPE)

Directory 'G:\My Drive\Food_Recognition\Datasets\export_dir' already exists; export will be merged with existing files
 100% |███████| 119/119 [5.7s elapsed, 0s remaining, 18.4 samples/s]      


### CAREFULLY: Clear dataset

In [94]:
samples_dicts = ds.to_dict()['samples']
for sample in samples_dicts:
    if TAG in sample['tags']:
        samples_dicts.remove(sample)
# HOW TO REPLACE DATASET BY THIS DICT ?

 100% |█████████| 46/46 [140.9ms elapsed, 0s remaining, 326.5 samples/s]     
